#Equal-Weight S&P 500 screener

In [ ]:
!pip install numpy
!pip install pandas
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 3.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math

In [ ]:
stocks = pd.read_csv('sp_500_stocks.csv')
type(stocks)
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [ ]:
IEX_CLOUD_API_TOKEN = 'your_IEX_cloud_api_token'

In [ ]:
symbol = 'AAPL'
api_url = f''your_IEX_sandbox_api_link''
#print(api_url)
print(requests.get(api_url).text)
data = requests.get(api_url).json()
#type(data)
print(data)

In [ ]:
price = data['larestPrice']
market_cap = data['marketCap']
print(price)
print(market_cap)

In [ ]:
my_columns = ['ticker', 'stock prices', 'market capitalization', 'number of shares to buy']
final_dataFrame = pd.DataFrame(columns = my_columns)
final_dataFrame

,ticker,stock prices,market capitalization,number of shares to buy


In [ ]:
final_dataFrame.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
        index = my_columns
    ),
    ignore_index=True

)

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['ticker']:
  api_url = f'your_IEX_sandbox_api_link'
  data = requests.get(api_url).json()
  final_dataframe = final_dataframe.append(
      [
      stocks,
      data['latestprice'],
      data['marketcap'],
      'N/A',
      ],
      index = my_columns
  ),
  ignore_index = True

In [ ]:
final_dataframe

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks['tickers'],100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'your_IEX_sandbox_api_link'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol,
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['marketCap'],
                                                   'N/A'],
                                                  index = my_columns),
                                        ignore_index = True)


final_dataframe

In [ ]:
portfolio_size = input('Enter the size of your portfollio? ')
try:
  val = float(portfolio_size)
  print(val)
except ValueError:
  print("That's not a number! \n Please try again: ")
  portfolio_size = input('Enter the size of your portfollio? ')
  val = float(portfolio_size)


Enter the size of your portfollio? hi
That's not a number! 
 Please try again: 
Enter the size of your portfollio? 56


In [ ]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
  final_dataframe.loc[i, "Number of stocks to buy: "] = math.floor(position_size/final_dataframe.loc[i, 'stock_price'])
  final_dataframe

In [ ]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [ ]:
writer.save()